# Automated Hyper-Parameter Tuned Baseline Classifiers for Benchmark


### Features Extracted [Mel(128), MFCC(40), Chroma(12)]
### Gender Splitted
### Hyper-Parameter Tuning
### Classical Classifiers and Voting Classifiers

* This notebook extracts audio features and considers the following nine baseline classifiers which will serve as a benchmark for the proposed model
1. Light Gradient Boosing Machine
2. Random Forest
3. eXtreme Gradient Boosting
3. Multi-Layer Perceptron
4. K-Nearest Neighbor
5. Decision Tree
6. Logistic Regression
* The notebook also combines the best of classifiers and creates four new classifiers which is a combination of best of the above classifiers
7. V1 [Multi-Layer Perceptron, Light Gradient Boosing Machine]
8. V2 [K-Nearest Neighbor, eXtreme Gradient Boosting, Multi-Layer Perceptron]
9. V3 [eXtreme Gradient Boosting, Multi-Layer Perceptron, Random Forest, Logistic Regression]
10. V4 [Multi-Layer Perceptron, eXtreme Gradient Boosting]

* The hyper-parementer tuning is performed using Optune Framework (https://optuna.org/) to get the best parameters for the baseline classifiers
* The baseline classifiers are cross-validated and the classification report and confusion matrix is displayed

In [1]:
# Activating ser conda environment
! source activate ser

# Installing the Optune Framework package for Hyper-Paramenter Tuning
! pip install --user optuna

# Installing other classifier packages
! pip install --user lightgbm


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Notebook Settings

# Supress warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Loading Libraries

import pandas as pd
import numpy as np
import os
import random
import sys
import glob 
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.multiclass import unique_labels
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

import lightgbm as lgb
import xgboost as xgb
import optuna
from tqdm import tqdm

In [4]:
# Function to extract audio features

def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft=np.abs(librosa.stft(X))
    result=np.array([])
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    result=np.hstack((result, mfccs))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
    return result

In [5]:
# Declaring Classes and Gender

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

def gender(g):
    """Returns Gender Label"""
    if int(g[0:2]) % 2 == 0:
        return 'female'
    else:
        return 'male'

In [6]:
# Function to load data

def load_data(test_size=0.2):
    """Loads Data from directory containing WAV files."""
    x,y=[],[]
    for file in tqdm(glob.glob("./DATASET/*.wav")): # Path to audio dataset
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]] + '_' + gender(file_name.split("-")[-1])
        feature=extract_feature(file)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [7]:
# Loading Data

X_train, X_test, y_train, y_test = load_data()

100%|██████████| 5252/5252 [12:53<00:00,  6.79it/s]


In [8]:
print((X_train.shape[0], X_test.shape[0]))
print((X_train.shape[1], X_test.shape[1]))
print(f'Features extracted: {X_train.shape[1]}')

(4201, 1051)
(180, 180)
Features extracted: 180


## HYPER-PARAMETER TUNING USING OPTUNA FRAMEWORK (OPTIONAL)

N.B.: If hyper-parameters are obtained, copy them and paste it into the corrosponding model parameters below and do not run this cell

### LGB

In [9]:
def objective_lgb(trial): 

    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 150),
        'max_depth': trial.suggest_int('max_depth', 2, 100),
        'n_estimators': trial.suggest_int('n_estimators', 10, 20000),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 100000, 500000),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 500),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 100),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.0, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-0),
        'boosting_type': trial.suggest_categorical('boosting_type', ['goss','gbdt','dart']),
        'objective': 'multiclass',
        'verbose': -1,
        'random_state':22,
        }
   
    model = lgb.LGBMClassifier(**params, n_jobs = 12) 
    
    model.set_params(**params)

    return np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))

In [10]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_lgb, n_trials=25)

[I 2022-12-28 12:40:49,820] A new study created in memory with name: no-name-2b577db4-2701-456e-9182-9adf236a1f39


[LightGBM] [Warning] min_data_in_leaf is set=467, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=467
[LightGBM] [Warning] min_data_in_leaf is set=467, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=467
[LightGBM] [Warning] min_data_in_leaf is set=467, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=467
[LightGBM] [Warning] min_data_in_leaf is set=467, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=467
[LightGBM] [Warning] min_data_in_leaf is set=467, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=467


[I 2022-12-28 12:42:16,093] Trial 0 finished with value: 0.7307748711850971 and parameters: {'num_leaves': 15, 'max_depth': 18, 'n_estimators': 9423, 'subsample_for_bin': 177318, 'min_data_in_leaf': 467, 'reg_alpha': 21.364324863046125, 'colsample_bytree': 0.4166169833515284, 'learning_rate': 0.08988880633472562, 'boosting_type': 'gbdt'}. Best is trial 0 with value: 0.7307748711850971.


[LightGBM] [Warning] min_data_in_leaf is set=439, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=439
[LightGBM] [Warning] min_data_in_leaf is set=439, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=439
[LightGBM] [Warning] min_data_in_leaf is set=439, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=439
[LightGBM] [Warning] min_data_in_leaf is set=439, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=439
[LightGBM] [Warning] min_data_in_leaf is set=439, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=439


[I 2022-12-28 12:45:58,324] Trial 1 finished with value: 0.659364135666157 and parameters: {'num_leaves': 8, 'max_depth': 98, 'n_estimators': 12899, 'subsample_for_bin': 269886, 'min_data_in_leaf': 439, 'reg_alpha': 12.73644948176721, 'colsample_bytree': 0.18931542811418567, 'learning_rate': 7.905719250121463e-05, 'boosting_type': 'goss'}. Best is trial 0 with value: 0.7307748711850971.


...

[I 2022-12-28 15:10:48,396] Trial 22 finished with value: 0.8167077175697866 and parameters: {'num_leaves': 102, 'max_depth': 55, 'n_estimators': 5239, 'subsample_for_bin': 451297, 'min_data_in_leaf': 179, 'reg_alpha': 10.166085786278504, 'colsample_bytree': 0.5564115021925584, 'learning_rate': 0.23410649113166168, 'boosting_type': 'goss'}. Best is trial 22 with value: 0.8167077175697866.


[LightGBM] [Warning] min_data_in_leaf is set=171, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=171
[LightGBM] [Warning] min_data_in_leaf is set=171, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=171
[LightGBM] [Warning] min_data_in_leaf is set=171, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=171
[LightGBM] [Warning] min_data_in_leaf is set=171, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=171
[LightGBM] [Warning] min_data_in_leaf is set=171, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=171


[I 2022-12-28 15:11:21,823] Trial 23 finished with value: 0.8112335088613328 and parameters: {'num_leaves': 99, 'max_depth': 54, 'n_estimators': 4439, 'subsample_for_bin': 449828, 'min_data_in_leaf': 171, 'reg_alpha': 10.298161147968209, 'colsample_bytree': 0.4732585790389039, 'learning_rate': 0.22318569716533146, 'boosting_type': 'goss'}. Best is trial 22 with value: 0.8167077175697866.


[LightGBM] [Warning] min_data_in_leaf is set=282, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=282
[LightGBM] [Warning] min_data_in_leaf is set=282, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=282
[LightGBM] [Warning] min_data_in_leaf is set=282, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=282
[LightGBM] [Warning] min_data_in_leaf is set=282, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=282
[LightGBM] [Warning] min_data_in_leaf is set=282, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=282


[I 2022-12-28 15:11:32,441] Trial 24 finished with value: 0.32983353151010697 and parameters: {'num_leaves': 63, 'max_depth': 40, 'n_estimators': 1674, 'subsample_for_bin': 410277, 'min_data_in_leaf': 282, 'reg_alpha': 5.822028287321087, 'colsample_bytree': 0.24351807574820955, 'learning_rate': 0.9590454530172389, 'boosting_type': 'goss'}. Best is trial 22 with value: 0.8167077175697866.


### RF

In [12]:
def objective_rf(trial):
    
    params = {
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 5, 25),
        'n_estimators': trial.suggest_int('n_estimators', 15000, 25000),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 5),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 25),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 50, 250),
        }
   
    model = RandomForestClassifier(**params, random_state = 22) 
    
    model.set_params(**params)

    return np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_rf, n_trials=25)

[I 2022-12-29 00:44:47,370] A new study created in memory with name: no-name-72c7325e-1c5e-4c94-b8ae-e3c303489eb7


### XGB

In [ ]:
def objective_xgb(trial):

    param = {
        'silent': 1,
        'objective': 'multi:softmax',
        'num_class': 16,
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0)
    }

    if param['booster'] == 'gbtree' or param['booster'] == 'gblinear':
        param['max_depth'] = trial.suggest_int('max_depth', 1, 9)
        param['eta'] = trial.suggest_loguniform('eta', 1e-8, 1.0)
        param['gamma'] = trial.suggest_loguniform('gamma', 1e-8, 1.0)
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
    if param['booster'] == 'dart':
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_loguniform('rate_drop', 1e-8, 1.0)
        param['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)
  
    model = xgb.XGBClassifier() 
    
    model.set_params(**param)

    return np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_xgb, n_trials=25)

### MLP

In [24]:
def objective_mlp(trial):

    params = {
        'activation': trial.suggest_categorical('activation', ['logistic', 'tanh', 'relu']),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'sgd', 'adam']),
        'hidden_layer_sizes':trial.suggest_int('hidden_layer_sizes', 100, 1500),
        'alpha': trial.suggest_uniform('alpha', 0.001, 0.99),
        'batch_size':trial.suggest_int('batch_size', 150, 300), 
        'learning_rate': trial.suggest_categorical('learning_rate', ['adaptive', 'constant', 'invscaling']),
        'max_iter': 1000
        }
  
    model = MLPClassifier(**params, random_state = 22) 
    
    model.set_params(**params)

    return np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))

In [25]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_mlp, n_trials=25)

[I 2022-12-28 18:31:35,583] A new study created in memory with name: no-name-bba9d8bb-d87f-4926-b67d-f6617224f1a7
[I 2022-12-28 18:35:19,671] Trial 0 finished with value: 0.7272133514523527 and parameters: {'activation': 'tanh', 'solver': 'lbfgs', 'hidden_layer_sizes': 272, 'alpha': 0.6655292519385918, 'batch_size': 249, 'learning_rate': 'constant'}. Best is trial 0 with value: 0.7272133514523527.
[I 2022-12-28 18:35:49,600] Trial 1 finished with value: 0.807188437800804 and parameters: {'activation': 'logistic', 'solver': 'adam', 'hidden_layer_sizes': 166, 'alpha': 0.721368233089467, 'batch_size': 151, 'learning_rate': 'adaptive'}. Best is trial 1 with value: 0.807188437800804.
[I 2022-12-28 18:36:47,642] Trial 2 finished with value: 0.30560330672102376 and parameters: {'activation': 'relu', 'solver': 'sgd', 'hidden_layer_sizes': 129, 'alpha': 0.3024295887507661, 'batch_size': 166, 'learning_rate': 'adaptive'}. Best is trial 1 with value: 0.807188437800804.
[I 2022-12-28 18:37:42,417]

### KNN

In [9]:
def objective_knn(trial):

    params = { 
        'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
        'n_neighbors': trial.suggest_int('n_neighbors', 16, 200),
        }

    model = KNeighborsClassifier(**params) 
    
    model.set_params(**params)

    return np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))

In [10]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_knn, n_trials=25)

[I 2022-12-28 22:46:42,479] A new study created in memory with name: no-name-1e04c718-ed05-46bf-908e-04be5aec2405
[I 2022-12-28 22:46:43,440] Trial 0 finished with value: 0.6146073268784328 and parameters: {'weights': 'uniform', 'n_neighbors': 144}. Best is trial 0 with value: 0.6146073268784328.
[I 2022-12-28 22:46:43,647] Trial 1 finished with value: 0.6634077911783024 and parameters: {'weights': 'distance', 'n_neighbors': 120}. Best is trial 1 with value: 0.6634077911783024.
[I 2022-12-28 22:46:43,942] Trial 2 finished with value: 0.6003252930185154 and parameters: {'weights': 'uniform', 'n_neighbors': 181}. Best is trial 1 with value: 0.6634077911783024.
[I 2022-12-28 22:46:44,215] Trial 3 finished with value: 0.6172263744974804 and parameters: {'weights': 'uniform', 'n_neighbors': 136}. Best is trial 1 with value: 0.6634077911783024.
[I 2022-12-28 22:46:44,511] Trial 4 finished with value: 0.6022297718136006 and parameters: {'weights': 'uniform', 'n_neighbors': 178}. Best is trial

### DT

In [28]:
def objective_dt(trial):
    
    params = {
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 5, 100),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 5),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 25),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 50, 250),
        }
    
    model = DecisionTreeClassifier(**params, random_state = 22) 
    
    model.set_params(**params)

    return np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))

In [29]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_dt, n_trials=25)

[I 2022-12-28 19:46:06,019] A new study created in memory with name: no-name-d8bf2aa8-bbb5-4285-8748-b1255aeae45e
[I 2022-12-28 19:46:08,946] Trial 0 finished with value: 0.6500787044901195 and parameters: {'criterion': 'gini', 'max_depth': 32, 'min_samples_leaf': 4, 'min_samples_split': 15, 'max_leaf_nodes': 77}. Best is trial 0 with value: 0.6500787044901195.
[I 2022-12-28 19:46:15,053] Trial 1 finished with value: 0.6679332993601721 and parameters: {'criterion': 'entropy', 'max_depth': 93, 'min_samples_leaf': 5, 'min_samples_split': 19, 'max_leaf_nodes': 105}. Best is trial 1 with value: 0.6679332993601721.
[I 2022-12-28 19:46:21,249] Trial 2 finished with value: 0.6674588075420418 and parameters: {'criterion': 'entropy', 'max_depth': 61, 'min_samples_leaf': 4, 'min_samples_split': 21, 'max_leaf_nodes': 207}. Best is trial 1 with value: 0.6679332993601721.
[I 2022-12-28 19:46:27,408] Trial 3 finished with value: 0.6617448049374327 and parameters: {'criterion': 'entropy', 'max_depth'

### LR

In [ ]:
def objective_lr(trial):

    params = { 
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'solver': trial.suggest_categorical('solver', ['newton-cg', 'sag', 'saga', 'lbfgs']),
        'multi_class':'multinomial',
        'max_iter': 5000
        }
    
    model = LogisticRegression(**params, random_state = 22) 
    
    model.set_params(**params)

    return np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_lr, n_trials=10)

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Hyper-Tuned Parameters

In [10]:
lgb_params = {'num_leaves': 102, 
              'max_depth': 55, 
              'n_estimators': 5239, 
              'subsample_for_bin': 451297, 
              'min_data_in_leaf': 179, 
              'reg_alpha': 10.166085786278504, 
              'colsample_bytree': 0.24351807574820955, 
              'learning_rate': 0.9590454530172389, 
              'boosting_type': 'goss'}

In [11]:
rf_params = {'criterion': 'entropy', 
             'max_depth': 15, 
             'n_estimators': 22984, 
             'min_samples_leaf': 3, 
             'min_samples_split': 9, 
             'max_leaf_nodes': 239, 
             'random_state': 22}

In [12]:
xgb_params = {'booster': 'gbtree', 
              'lambda': 7.201651687969849e-08, 
              'alpha': 2.2495125443474775e-05, 
              'max_depth': 7, 
              'eta': 9.307925211476325e-06, 
              'gamma': 1.7948741419263195e-05, 
              'grow_policy': 'lossguide'}

In [13]:
dt_params = {'criterion': 'gini', 
             'max_depth': 77, 
             'min_samples_leaf': 2, 
             'min_samples_split': 8, 
             'max_leaf_nodes': 229}

In [14]:
mlp_params = {'activation': 'tanh', 
              'solver': 'adam', 
              'hidden_layer_sizes': 1442, 
              'alpha': 0.0027099149530944583, 
              'batch_size': 298, 
              'learning_rate': 'invscaling',
              'max_iter':1000}

In [15]:
knn_params = {'weights': 'distance', 
              'n_neighbors': 16}

In [16]:
lr_params = {'multi_class':'multinomial',
             'class_weight': None, 
             'solver': 'saga', 
             'max_iter':10000}

In [17]:
models = {'dt':DecisionTreeClassifier(**dt_params),
          'rf':RandomForestClassifier(**rf_params), 
          'lgb':lgb.LGBMClassifier(**lgb_params), 
          'xgb':xgb.XGBClassifier(**xgb_params),
          'mlp':MLPClassifier(**mlp_params), 
          'kn':KNeighborsClassifier(**knn_params),  
          'lr':LogisticRegression(**lr_params)
         }

model_abrv = {'dt':'Decision Tree Classifier', 
              'rf':'Random Forest Classifier', 
              'lgb':'LGBM Classifier', 
              'xgb':'XGB Classifier', 
              'mlp':'MLP Classifier',
              'kn':'K-Nearest Neighbors', 
              'lr':'Logistic Regression', 
              'v':'Voting Classifier: MLP, LGB', 
              'v2':'Voting Classifier 2: KNN, XGB, MLP', 
              'v3':'Voting Classifier 3: XGB, MLP, RF, LR', 
              'v4':'Voting Classifier 4: MLP, XGB'
             }

### Voting Classifiers Hyper-Tuned Parameters

In [18]:
v_params = {'estimators':[('mlp', models['mlp']), 
                          ('lgb', models['lgb'])], 
            'voting':'soft'}

In [19]:
v2_params = {'estimators':[('kn', models['kn']), 
                           ('xgb', models['xgb']), 
                           ('mlp', models['mlp'])], 
             'voting':'soft'}

In [20]:
v3_params = {'estimators':[('xgb', models['xgb']),
                           ('mlp', models['mlp']),
                           ('rf', models['rf']), 
                           ('lr', models['lr'])], 
             'voting':'soft'}

In [21]:
v4_params = {'estimators':[('mlp', models['mlp']), 
                          ('xgb', models['xgb'])], 
            'voting':'soft'}

In [22]:
models['v'] = VotingClassifier(**v_params)
models['v2'] = VotingClassifier(**v2_params)
models['v3'] = VotingClassifier(**v3_params)
models['v4'] = VotingClassifier(**v4_params)

### Training and Metrices

In [23]:
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14, model='clf', save=True):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a seaborn heatmap. 
    Saves confusion matrix file to jpg file."""
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, ax=ax, fmt="d", cmap=plt.cm.Oranges)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
        
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    # fix for mpl bug that cuts off top/bottom of seaborn viz
    b, t = plt.ylim() 
    b += 0.5 
    t -= 0.5 
    plt.ylim(b, t) 
    if save == True:
        plt.savefig('tuned_' + model_abrv[model] + '_confusion_matrix.png')
    plt.show()

In [24]:
def model(clf, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, models=models, save=False, print_stat=True, inc_train=False, cv=False):
    """Trains models and outputs score metrics. Takes an identifier, list of models, and split dataset as inputs and has options for saving model, 
    printing confusion matrix and classification report and getting cross-validated 5 fold accuracy."""
    clf_model = models[clf]
    clf_model.fit(X_train, y_train)
    y_pred = clf_model.predict(X_test)
    if print_stat == True:
        clf_report = pd.DataFrame(classification_report(y_test,y_pred, output_dict=True)).T
        clf_report.to_csv('tuned_' + model_abrv[clf] + '_classification_report.csv')
        print(model_abrv[clf])
        print('\nTest Stats\n', classification_report(y_test,y_pred))
        print_confusion_matrix(confusion_matrix(y_test, y_pred), unique_labels(y_test, y_pred), model=clf)
        if inc_train == True:
            print(model_abrv[clf])
            print('\nTrain Stats\n', classification_report(y_train,clf_model.predict(X_train)))
            print_confusion_matrix(confusion_matrix(y_train, clf_model.predict(X_train)), unique_labels(y_test, y_pred), model=clf)
    if cv == True:
        print(model_abrv[clf] + ' CV Accuracy:',  
              np.mean(cross_val_score(clf_model, X_train, y_train, cv=5, scoring='accuracy')))
    if save == True:
        return clf_model

In [ ]:
# Training and 5-Fold Cross Validation

for key in models.keys():
    model(key, cv=True, print_stat=False)

Decision Tree Classifier CV Accuracy: 0.6765047279316007


In [ ]:
# Model Types

print(models.keys())

In [ ]:
# Results and Metrices

for key in models.keys():
    model(key, inc_train=True)